In [ ]:
import numpy as np

In [ ]:
NUMBERS = 26

ACTIONS = {
    "N": np.array([0, -1, 0]),
    "S": np.array([0, 1, 0]),
    "E": np.array([0, 0, 1]),
    "W": np.array([0, 0, -1]),
    "U": np.array([-1, 0, 0]),
    "D": np.array([1, 0, 0]),
}

class State:
    """
    Constructor for the State class

    Args:
        state (list[list]): the state of the node
        action (str, optional): the action taken to get to this node. Defaults to None.
        parent (State, optional): the parent node. Defaults to None.
        cost (int, optional): the cost to get to this node. Defaults to 0.
    """
    def __init__(self, state : list[list], action : str = None, parent : "State" = None, cost : int = 0):
        self.state = np.reshape(state, (3, 3, 3)) # represents the state of each node (3x3x3 list)
        self.parent = parent # represents the parent node
        self.action = action # represents the action taken to get to this node
        self.cost = cost # represents g(n)
        self.heuristic = None # represents h(n)
        self.total_cost = None # represents f(n)

    """
    Locates the blank position in the state

    Args:
        value (int, str): the value to be located in the state

    Returns:
        np.array: the location of the value in the state

    """
    def locate(self, value : [int, str]) -> np.array:  # will be used to locate the blank position
        return np.array(np.where(self.state == str(value))).T[0]

    """
    Finds all the legal actions that can be taken from the current state.

    Returns:
        list: all the legal actions that can be taken from the current state
    """
    def legal_actions(self) -> list:
        blank_location = self.locate('*')

        actions = []

        for action in ACTIONS.keys():
            new_location = blank_location + ACTIONS[action]

            # check if the new location is within the bounds of the state
            if not np.any((new_location < 0) | (new_location > 2)):
                actions.append(action)

        return actions

    """
    Performs the action on the current state and returns the new state.

    Args:
        action (str): the action to be taken

    Returns:
        State: the new state after the action has been taken
    """
    def slide(self, action : str) -> "State":
        blank_location = self.locate('*')
        new_location = blank_location + ACTIONS[action]

        new_state = self.state.copy()
        new_state[tuple(blank_location)] = new_state[tuple(new_location)]
        new_state[tuple(new_location)] = "*"

        return State(new_state, action, self, self.cost + 1)

    """
    Calculates the heuristic of the current state using Manhattan distance.

    Args:
        goal (State): the goal state
    """
    def calculate_heuristic(self, goal : "State") -> None:
        self.heuristic = sum([np.abs(self.locate(str(i)) - goal.locate(str(i))).sum() for i in range(1, NUMBERS + 1)])
        self.total_cost = self.cost + self.heuristic

    """
    Checks if the current state is equal to the goal state.

    Args:
        goal (State): the goal state
    """
    def __eq__(self, other) -> bool:
        return np.array_equal(self.state, other.state)

    def __repr__(self) -> str:
        string = ''

        for z in range(3):
            for y in range(3):
                for x in range(3):
                    string += self.state[z][y][x] + ' '

                string = string.strip()
                string += '\n'

            string += '\n'

        return string

In [ ]:
with open('test.txt', 'r') as file:
    initial_state = []
    goal_state = []

    for _ in range(3):
        for _ in range(3):
            initial_state.append(file.readline().strip('\n').split(' '))
        file.readline()

    for _ in range(3):
        for _ in range(3):
            goal_state.append(file.readline().strip('\n').split(' '))
        file.readline()

In [ ]:
initial_state = State(initial_state)
GOAL = State(goal_state)

In [ ]:
initial_state.calculate_heuristic(GOAL)
GOAL.calculate_heuristic(GOAL)

In [ ]:
"""
A* search algorithm

Args:
    initial_state (State): the initial state
    goal (State): the goal state

Returns:
    list: the solution States from the initial state to the goal state
    generated_nodes (int): the number of nodes generated
"""
def a_star(initial_state : State, goal : State) -> "list[State]":
    frontier = [initial_state]
    explored = []
    generated_nodes = 1

    # while frontier is not empty
    while frontier:
        frontier.sort(key = lambda x: x.total_cost)
        state = frontier.pop(0)

        # if goal state is reached
        if state == goal:
            solution = []
            while state.parent:
                solution.append(state)
                state = state.parent

            return solution[::-1], generated_nodes

        explored.append(state)

        for action in state.legal_actions():
            child = state.slide(action)

            if child not in explored and child not in frontier:
                generated_nodes += 1
                child.calculate_heuristic(goal)
                frontier.append(child)

            elif child in frontier:
                if child.cost < frontier[frontier.index(child)].cost:
                    frontier.remove(child)
                    frontier.append(child)

    return None

In [ ]:
def create_test_case():
    state = initial_state
    actions = []

    with open('test.txt', 'w') as test_file:
        test_file.write(initial_state.__repr__())

        move_count = np.random.randint(0, 50)
        print("Number of moves: " + str(move_count))

        for _ in range(move_count):
            action = np.random.choice(state.legal_actions())
            actions.append(action)
            state = state.slide(action)

        print('Actions:', ' '.join(actions))

        test_file.write(state.__repr__())

In [ ]:
create_test_case()

Number of moves: 34
Actions: N N S S W W N N E U D E U U S S W D E W U W D U N E W S N S D N U N


In [ ]:
solution, generated_nodes = a_star(initial_state, GOAL)

with open('output.txt', 'w') as output_file:
    with open('input.txt', 'r') as input_file:
        for line in input_file.readlines():
            output_file.write(line)

    output_file.write('\n')
    output_file.write('\n')

    output_file.write(str(max(len(solution) - 1, 0)))
    output_file.write('\n')
    output_file.write(str(generated_nodes))
    output_file.write('\n')
    output_file.write(' '.join([state.action for state in solution]))
    output_file.write('\n')
    output_file.write(str(initial_state.total_cost) + ' ' + ' '.join([str(state.total_cost) for state in solution]))


In [ ]:
solution[0].total_cost

16